<h1><center> Project 1 - Twitter API data gathering

In [4]:
import pandas as pd
import numpy as np
import requests
import tweepy
import time
from authentication import consumer_key, consumer_secret, access_token, access_secret

#### 1) Lets authenticate into tweepy

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

#### 2) Get the list of tweets' ids

In [6]:
twitter = pd.read_csv('twitter-archive-enhanced.csv')

In [7]:
twitter_ids =  list(twitter.tweet_id.values)

#### 3) Get the data from twitter API

In [8]:
class DataGatherAPI():
    '''Gathers data form twitter API and waits for the rate limits'''

    tweets_dict = {}
    exceptions = []
    twitter_ids_rate_exceeded = []
    end = 0
    
    def __init__(self, rate_allowance=900):
        '''Creates attributes for a class'''
        
        self.rate_limits = {
            'rate_start_time': time.time(),
            'rate_usage': 0
        }
        self.rate_allowance=rate_allowance
        
    def _rate_limit_helper(self):
        '''Created to eliminate repetition of code, helper to check_rate_limit func'''
        
        DataGatherAPI.end=time.time()
        elapsed = DataGatherAPI.end - self.rate_limits['rate_start_time']

        if elapsed <900:
            wait = (900-elapsed)
            end_time = str(time.asctime( time.localtime(wait + time.time())))[11:19]
            print('Need to wait '+str(int(wait)/60)[:4]+' minutes; End time: '+ end_time)
            time.sleep(wait)
        
        self.rate_limits['rate_usage']=0
        self.rate_limits['rate_start_time']=time.time()
        
    def check_rate_limit(self):
        '''Checks if rate limits have been reached'''
        
        print('Current rate usage is: '+ str(self.rate_limits['rate_usage'])+'/'+str(self.rate_allowance))
        if self.rate_allowance==0:
            print('You need to use rate limits')
            raise ValueError('You need to use rat elimits for this, please specify them')
        else:
            if self.rate_limits['rate_usage']==self.rate_allowance:
                self._rate_limit_helper()
        
        
    def gather_data(self, twitter_ids):
        '''Uses tweepy API scrape data'''
        
        for number, idx in enumerate(twitter_ids):
            if idx not in DataGatherAPI.tweets_dict:
                DataGatherAPI.tweets_dict[idx] = {}

            self.check_rate_limit()

            try:
                status = api.get_status(idx)


                favorite_count = status.favorite_count
                retweet_count = status.retweet_count

                # storing the data
                DataGatherAPI.tweets_dict[idx]['favorite_count'] = favorite_count
                DataGatherAPI.tweets_dict[idx]['retweet_count'] = retweet_count

                print(str(number+1) +'/' + str(len(twitter_ids)),
                      DataGatherAPI.tweets_dict[idx]['favorite_count'], 
                      DataGatherAPI.tweets_dict[idx]['retweet_count'], sep = '; ')

            except Exception as e:
                
                DataGatherAPI.exceptions.append((idx, e))
                print(e, 'for id:', idx)
                
                if '429 Too Many Requests\n88 - Rate limit exceeded' in str(e):
                    DataGatherAPI.twitter_ids_rate_exceeded.append(idx)
                    self._rate_limit_helper()
                else:
                    DataGatherAPI.tweets_dict[idx]['favorite_count'] = None
                    DataGatherAPI.tweets_dict[idx]['retweet_count'] = None
                    
                
            self.rate_limits['rate_usage'] += 2

Probably the easier solution would have been to just set: wait_on_rate_limit and wait_on_rate_limit_notify to True

In [8]:
d = DataGatherAPI()
d.gather_data(twitter_ids)
d.gather_data(DataGatherAPI.twitter_ids_rate_exceeded)

Current rate usage is: 0/900
1/2356; 33702; 6976
Current rate usage is: 2/900
2/2356; 29231; 5278
Current rate usage is: 4/900
3/2356; 21975; 3466
Current rate usage is: 6/900
4/2356; 36778; 7190
Current rate usage is: 8/900
5/2356; 35164; 7723
Current rate usage is: 10/900
6/2356; 17739; 2586
Current rate usage is: 12/900
7/2356; 10325; 1649
Current rate usage is: 14/900
8/2356; 56643; 15692
Current rate usage is: 16/900
9/2356; 24436; 3605
Current rate usage is: 18/900
10/2356; 27846; 6075
Current rate usage is: 20/900
11/2356; 26954; 6119
Current rate usage is: 22/900
12/2356; 24496; 4142
Current rate usage is: 24/900
13/2356; 41871; 8309
Current rate usage is: 26/900
14/2356; 23574; 3701
Current rate usage is: 28/900
15/2356; 13307; 1874
Current rate usage is: 30/900
16/2356; 22014; 4426
Current rate usage is: 32/900
17/2356; 25522; 3746
Current rate usage is: 34/900
18/2356; 22385; 3517
Current rate usage is: 36/900
19/2356; 17253; 2867
Current rate usage is: 38/900
404 Not Found


154/2356; 18213; 4232
Current rate usage is: 308/900
155/2356; 66001; 19920
Current rate usage is: 310/900
404 Not Found
144 - No status found with that ID. for id: 861769973181624320
Current rate usage is: 312/900
157/2356; 32195; 9236
Current rate usage is: 314/900
158/2356; 15311; 3577
Current rate usage is: 316/900
159/2356; 14957; 3322
Current rate usage is: 318/900
160/2356; 0; 1850
Current rate usage is: 320/900
161/2356; 0; 676
Current rate usage is: 322/900
162/2356; 6705; 1850
Current rate usage is: 324/900
163/2356; 21185; 4588
Current rate usage is: 326/900
164/2356; 16531; 3057
Current rate usage is: 328/900
165/2356; 14969; 5107
Current rate usage is: 330/900
166/2356; 0; 26635
Current rate usage is: 332/900
167/2356; 17573; 3573
Current rate usage is: 334/900
168/2356; 13948; 3061
Current rate usage is: 336/900
169/2356; 16788; 1361
Current rate usage is: 338/900
170/2356; 80181; 26187
Current rate usage is: 340/900
171/2356; 30332; 11927
Current rate usage is: 342/900
1

301/2356; 18084; 4257
Current rate usage is: 602/900
302/2356; 11811; 2021
Current rate usage is: 604/900
303/2356; 0; 507
Current rate usage is: 606/900
304/2356; 0; 25032
Current rate usage is: 608/900
305/2356; 13883; 2745
Current rate usage is: 610/900
306/2356; 19909; 3976
Current rate usage is: 612/900
307/2356; 17851; 3883
Current rate usage is: 614/900
308/2356; 0; 7404
Current rate usage is: 616/900
309/2356; 16620; 3293
Current rate usage is: 618/900
310/2356; 0; 1540
Current rate usage is: 620/900
311/2356; 0; 19126
Current rate usage is: 622/900
312/2356; 15404; 2734
Current rate usage is: 624/900
313/2356; 7220; 1540
Current rate usage is: 626/900
314/2356; 1988; 63
Current rate usage is: 628/900
315/2356; 24522; 5288
Current rate usage is: 630/900
316/2356; 20892; 2755
Current rate usage is: 632/900
317/2356; 10083; 1489
Current rate usage is: 634/900
318/2356; 20169; 5105
Current rate usage is: 636/900
319/2356; 12788; 2311
Current rate usage is: 638/900
320/2356; 0; 974

457/2356; 10376; 2074
Current rate usage is: 14/900
458/2356; 11590; 2361
Current rate usage is: 16/900
459/2356; 4594; 739
Current rate usage is: 18/900
460/2356; 48427; 25032
Current rate usage is: 20/900
461/2356; 10201; 2515
Current rate usage is: 22/900
462/2356; 11076; 2720
Current rate usage is: 24/900
463/2356; 0; 3132
Current rate usage is: 26/900
464/2356; 31746; 13539
Current rate usage is: 28/900
465/2356; 5280; 884
Current rate usage is: 30/900
466/2356; 0; 7834
Current rate usage is: 32/900
467/2356; 8284; 1858
Current rate usage is: 34/900
468/2356; 11503; 2406
Current rate usage is: 36/900
469/2356; 8108; 1510
Current rate usage is: 38/900
470/2356; 0; 4372
Current rate usage is: 40/900
471/2356; 9445; 1886
Current rate usage is: 42/900
472/2356; 9209; 2047
Current rate usage is: 44/900
473/2356; 28569; 7427
Current rate usage is: 46/900
474/2356; 8147; 1795
Current rate usage is: 48/900
475/2356; 8460; 1980
Current rate usage is: 50/900
476/2356; 0; 942
Current rate us

615/2356; 11172; 2820
Current rate usage is: 330/900
616/2356; 0; 6708
Current rate usage is: 332/900
617/2356; 7159; 1645
Current rate usage is: 334/900
618/2356; 10423; 3878
Current rate usage is: 336/900
619/2356; 0; 13324
Current rate usage is: 338/900
620/2356; 30401; 13324
Current rate usage is: 340/900
621/2356; 4724; 1680
Current rate usage is: 342/900
622/2356; 8615; 2244
Current rate usage is: 344/900
623/2356; 8005; 2158
Current rate usage is: 346/900
624/2356; 10239; 3492
Current rate usage is: 348/900
625/2356; 46621; 21891
Current rate usage is: 350/900
626/2356; 9464; 2677
Current rate usage is: 352/900
627/2356; 15303; 5079
Current rate usage is: 354/900
628/2356; 0; 3148
Current rate usage is: 356/900
629/2356; 9654; 2153
Current rate usage is: 358/900
630/2356; 0; 9632
Current rate usage is: 360/900
631/2356; 9101; 2529
Current rate usage is: 362/900
632/2356; 8743; 3103
Current rate usage is: 364/900
633/2356; 15863; 4557
Current rate usage is: 366/900
634/2356; 8748

...

1709/2356; 2598; 937
Current rate usage is: 718/900
1710/2356; 2408; 624
Current rate usage is: 720/900
1711/2356; 3294; 1186
Current rate usage is: 722/900
1712/2356; 3881; 1729
Current rate usage is: 724/900
1713/2356; 1571; 438
Current rate usage is: 726/900
1714/2356; 2340; 669
Current rate usage is: 728/900
1715/2356; 1321; 440
Current rate usage is: 730/900
1716/2356; 1355; 260
Current rate usage is: 732/900
1717/2356; 2580; 1042
Current rate usage is: 734/900
1718/2356; 1974; 441
Current rate usage is: 736/900
1719/2356; 3545; 1391
Current rate usage is: 738/900
1720/2356; 2130; 669
Current rate usage is: 740/900
1721/2356; 3309; 1557
Current rate usage is: 742/900
1722/2356; 2109; 854
Current rate usage is: 744/900
1723/2356; 2491; 822
Current rate usage is: 746/900
1724/2356; 3297; 1245
Current rate usage is: 748/900
1725/2356; 11707; 8124
Current rate usage is: 750/900
1726/2356; 2381; 771
Current rate usage is: 752/900
404 Not Found
144 - No status found with that ID. for id

1865/2356; 941; 192
Current rate usage is: 130/900
1866/2356; 29383; 14861
Current rate usage is: 132/900
1867/2356; 3521; 1998
Current rate usage is: 134/900
1868/2356; 2481; 1127
Current rate usage is: 136/900
1869/2356; 3168; 1382
Current rate usage is: 138/900
1870/2356; 5582; 2258
Current rate usage is: 140/900
1871/2356; 3348; 1496
Current rate usage is: 142/900
1872/2356; 843; 197
Current rate usage is: 144/900
1873/2356; 1052; 255
Current rate usage is: 146/900
1874/2356; 1912; 779
Current rate usage is: 148/900
1875/2356; 1164; 411
Current rate usage is: 150/900
1876/2356; 1735; 686
Current rate usage is: 152/900
1877/2356; 869; 213
Current rate usage is: 154/900
1878/2356; 2514; 996
Current rate usage is: 156/900
1879/2356; 929; 275
Current rate usage is: 158/900
1880/2356; 2408; 1034
Current rate usage is: 160/900
1881/2356; 879; 200
Current rate usage is: 162/900
1882/2356; 1390; 406
Current rate usage is: 164/900
1883/2356; 1993; 952
Current rate usage is: 166/900
1884/235

2024/2356; 7594; 3667
Current rate usage is: 448/900
2025/2356; 1169; 479
Current rate usage is: 450/900
2026/2356; 3096; 1205
Current rate usage is: 452/900
2027/2356; 1245; 570
Current rate usage is: 454/900
2028/2356; 1104; 478
Current rate usage is: 456/900
2029/2356; 992; 418
Current rate usage is: 458/900
2030/2356; 791; 371
Current rate usage is: 460/900
2031/2356; 6299; 2992
Current rate usage is: 462/900
2032/2356; 1048; 434
Current rate usage is: 464/900
2033/2356; 1492; 772
Current rate usage is: 466/900
2034/2356; 1203; 660
Current rate usage is: 468/900
2035/2356; 674; 200
Current rate usage is: 470/900
2036/2356; 1283; 652
Current rate usage is: 472/900
2037/2356; 7478; 3792
Current rate usage is: 474/900
2038/2356; 11368; 6302
Current rate usage is: 476/900
2039/2356; 808; 183
Current rate usage is: 478/900
2040/2356; 1202; 496
Current rate usage is: 480/900
2041/2356; 1725; 888
Current rate usage is: 482/900
2042/2356; 965; 483
Current rate usage is: 484/900
2043/2356; 

2184/2356; 593; 288
Current rate usage is: 768/900
2185/2356; 796; 389
Current rate usage is: 770/900
2186/2356; 477; 140
Current rate usage is: 772/900
2187/2356; 478; 258
Current rate usage is: 774/900
2188/2356; 700; 292
Current rate usage is: 776/900
2189/2356; 1151; 495
Current rate usage is: 778/900
2190/2356; 127; 18
Current rate usage is: 780/900
2191/2356; 618; 209
Current rate usage is: 782/900
2192/2356; 241; 63
Current rate usage is: 784/900
2193/2356; 227; 50
Current rate usage is: 786/900
2194/2356; 275; 87
Current rate usage is: 788/900
2195/2356; 354; 126
Current rate usage is: 790/900
2196/2356; 474; 267
Current rate usage is: 792/900
2197/2356; 409; 139
Current rate usage is: 794/900
2198/2356; 381; 112
Current rate usage is: 796/900
2199/2356; 491; 225
Current rate usage is: 798/900
2200/2356; 627; 325
Current rate usage is: 800/900
2201/2356; 468; 187
Current rate usage is: 802/900
2202/2356; 772; 457
Current rate usage is: 804/900
2203/2356; 779; 446
Current rate u

2346/2356; 394; 180
Current rate usage is: 192/900
2347/2356; 99; 47
Current rate usage is: 194/900
2348/2356; 247; 112
Current rate usage is: 196/900
2349/2356; 367; 196
Current rate usage is: 198/900
2350/2356; 1024; 700
Current rate usage is: 200/900
2351/2356; 115; 50
Current rate usage is: 202/900
2352/2356; 88; 36
Current rate usage is: 204/900
2353/2356; 245; 115
Current rate usage is: 206/900
2354/2356; 100; 36
Current rate usage is: 208/900
2355/2356; 112; 39
Current rate usage is: 210/900
2356/2356; 2283; 420


In [9]:
DataGatherAPI.tweets_dict

{892420643555336193: {'favorite_count': 33702, 'retweet_count': 6976},
 892177421306343426: {'favorite_count': 29231, 'retweet_count': 5278},
 891815181378084864: {'favorite_count': 21975, 'retweet_count': 3466},
 891689557279858688: {'favorite_count': 36778, 'retweet_count': 7190},
 891327558926688256: {'favorite_count': 35164, 'retweet_count': 7723},
 891087950875897856: {'favorite_count': 17739, 'retweet_count': 2586},
 890971913173991426: {'favorite_count': 10325, 'retweet_count': 1649},
 890729181411237888: {'favorite_count': 56643, 'retweet_count': 15692},
 890609185150312448: {'favorite_count': 24436, 'retweet_count': 3605},
 890240255349198849: {'favorite_count': 27846, 'retweet_count': 6075},
 890006608113172480: {'favorite_count': 26954, 'retweet_count': 6119},
 889880896479866881: {'favorite_count': 24496, 'retweet_count': 4142},
 889665388333682689: {'favorite_count': 41871, 'retweet_count': 8309},
 889638837579907072: {'favorite_count': 23574, 'retweet_count': 3701},
 8895

It seems like there are three types of errors:
<br> '404 Not Found\n144 - No status found with that ID.' - Tweet/account has been deleted
<br> '403 Forbidden\n179 - Sorry, you are not authorized to see this status.' - forbidden for some reason
<br> '429 Too Many Requests\n88 - Rate limit exceeded' - I need to wait some time to be able to gather that data due to the API rules 

#### 4) Storing the data

In [10]:
{str(a):b for (a,b) in DataGatherAPI.tweets_dict.items()}

{'892420643555336193': {'favorite_count': 33702, 'retweet_count': 6976},
 '892177421306343426': {'favorite_count': 29231, 'retweet_count': 5278},
 '891815181378084864': {'favorite_count': 21975, 'retweet_count': 3466},
 '891689557279858688': {'favorite_count': 36778, 'retweet_count': 7190},
 '891327558926688256': {'favorite_count': 35164, 'retweet_count': 7723},
 '891087950875897856': {'favorite_count': 17739, 'retweet_count': 2586},
 '890971913173991426': {'favorite_count': 10325, 'retweet_count': 1649},
 '890729181411237888': {'favorite_count': 56643, 'retweet_count': 15692},
 '890609185150312448': {'favorite_count': 24436, 'retweet_count': 3605},
 '890240255349198849': {'favorite_count': 27846, 'retweet_count': 6075},
 '890006608113172480': {'favorite_count': 26954, 'retweet_count': 6119},
 '889880896479866881': {'favorite_count': 24496, 'retweet_count': 4142},
 '889665388333682689': {'favorite_count': 41871, 'retweet_count': 8309},
 '889638837579907072': {'favorite_count': 23574, '

In [11]:
import json
twitter_json = json.dumps({str(a):b for (a,b) in DataGatherAPI.tweets_dict.items()})
with open(r'tweet_json.txt', 'w') as file:
    file.write(twitter_json)

In [12]:
with open(r'tweet_json.txt', 'r') as file:
    f = file.read()

In [13]:
pd.read_json(f).T

,favorite_count,retweet_count
1998-04-12 22:37:23.555336193,33702.0,6976.0
1998-04-10 03:03:41.306343426,29231.0,5278.0
1998-04-05 22:26:21.378084864,21975.0,3466.0
1998-04-04 11:32:37.279858688,36778.0,7190.0
1998-03-31 06:59:18.926688256,35164.0,7723.0
...,...,...
1991-02-08 21:40:48.165822465,88.0,36.0
1991-02-08 20:17:06.329800704,245.0,115.0
1991-02-08 17:16:52.701032449,100.0,36.0
1991-02-08 16:08:05.002620928,112.0,39.0
